In [1]:
import numpy as np
import scipy.io as scio
import scipy.sparse as scsp

In [2]:
tup = scio.mminfo('C:\\Users\\Petr\\Downloads\\Материал курсач\\cryg10K.mtx')#\\test.mtx')#\\onetone1\\onetone1.mtx')#\\add32.mtx')#
tup

(10000, 10000, 49699, 'coordinate', 'real', 'general')

In [3]:
A = scio.mmread('C:\\Users\\Petr\\Downloads\\Материал курсач\\cryg10K.mtx') #\\onetone1\\onetone1.mtx')#\\add32.mtx')#\\test.mtx')#
A

<10000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 49699 stored elements in COOrdinate format>

In [4]:
np.random.seed(abs(tup[0]-tup[2]))
b = A.dot(np.random.rand(tup[0]))
b, b.nbytes

(array([ -1.03039188e+04,   1.01131678e+04,  -6.97087869e+03, ...,
         -2.90448084e-03,  -2.13066032e-02,  -3.84251215e-03]), 80000)

In [115]:
%timeit A.toarray().dot(b) #!!!VERY BAD

110 ms ± 30.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
qq = scsp.coo.coo_matrix(qq)

In [35]:
%timeit qq.toarray()

21.6 ms ± 59 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [117]:
%timeit A.dot(b) #NICE

31.8 µs ± 50.2 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [12]:
def gmres(a, b, x0, m, tol = 1e-15):
    """Solves the linear system Ax = b with the GMRES(m)
    
    Parameters
    ----------
        a : ndarray or coo_matrix
            Square real unsymmetric matrix.
        b : ndarray
            Column vector.
        x0 : ndarray
            Initial guess.
        m : int
            Maximum size of Krylov subspace, m > 1.
        tol : float
            Tolerance for the stopping criterion: rorm(impicit residual) <= tol.
    
    Returns
    -------
        x : ndarray
            Approximate solution.
        in_res : ndarray
            Initial residual.
        im_res : ndarray
            Final implicit residual.
        n_mult : int
            Number of MV multiplications.
    """
    v = b - a.dot(x0)
    hh = np.linalg.norm(v)
    in_res = hh
    
    if hh <= tol:
        im_res = hh
        x = x0
        n_mult = 1
        return x, in_res, im_res, n_mult
    else:
        h = np.zeros((m,m))
        c = np.zeros((m,1))
        s = np.zeros((m,1))
        y = np.zeros((m+1,1))
        y[0] = hh
        j = 0
        im_res = hh
    
        while j < m and im_res > tol:
            
            v[j:] = v[j:]/hh
            vnew = a.dot(v[j:].T)
            h[0:j+1,j] = v.dot(vnew).T

            if(v.size == b.shape[0]):
                vnew -= v*h[0:j+1,j]
            else:
                vnew -= v.T.dot(h[0:j+1,j]).reshape((b.size,1))
            hh = np.linalg.norm(vnew)

            for i in range(0,j):
                h[i:i+2,j] = np.vstack((np.hstack((c[i],s[i])),np.hstack((-s[i],c[i])))).dot(h[i:i+2,j])

            if(hh!=0.0):
                ww=np.linalg.norm(np.hstack((h[j,j],hh)))
                c[j] = h[j,j]/ww
                s[j] = hh/ww
                y[j+1] = -s[j]*y[j]
                y[j] = c[j]*y[j]
                h[j, j] = ww
                im_res = float(abs(y[j+1]))
            else:
                im_res = 0.0

            j+=1
            v = np.vstack((v, vnew.T))
        j-=1
        y[j] = y[j]/h[j,j]
        for i in range(j-1,-1,-1):
            y[i] = (y[i] - h[i,i+1:j+1].dot(y[i+1:j+1]))/h[i,i]
        x = x0 + v[:j+1].T.dot(y[0:j+1]).T
        n_mult = j+1
        return x, in_res, im_res, n_mult

In [13]:
x0=np.zeros(b.size)
x,r0,rn,nm = gmres(A, b, x0, 1000, tol = 1e-15)
x

array([[ 0.67633749,  0.21345724,  0.79778533, ...,  0.01511855,
         0.01146739,  0.00271485]])

In [14]:
np.linalg.norm(rn)/np.linalg.norm(r0)

1.1968160804851386e-05

In [15]:
np.linalg.norm(rn-(b-A.dot(x[0])))/np.linalg.norm(b-A.dot(x[0]))

5.423407294990354

In [16]:
b-A.dot(x[0])

array([-0.01680314, -0.03588657,  0.02564902, ..., -0.00303412,
        0.00337383, -0.00281881])